In [1]:
from face2voice.models.SpeakerEncoder import SpeakerEncoder
from openvoice.api import ToneColorConverter
import torch
import numpy as np
from face2voice.models.Face2Voice import Face2VoiceModel
from face2voice.models.FaceEncoder import FaceEncoder
from PIL import Image
from torchvision import transforms
from face2voice.losses.compound_loss import CompoundLoss

c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
tone_color_converter = ToneColorConverter(r'face2voice\checkpoints\tone_conv\config.json', device="cpu")
tone_color_converter.load_ckpt(r'face2voice\checkpoints\tone_conv\checkpoint.pth')

c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'face2voice\checkpoints\tone_conv\checkpoint.pth'
missing/unexpected keys: [] []


In [3]:
speaker_encoder = SpeakerEncoder(config_path=r'face2voice\checkpoints\tone_conv\config.json', ckpt_path=r'face2voice\checkpoints\tone_conv\checkpoint.pth')

Loaded checkpoint 'face2voice\checkpoints\tone_conv\checkpoint.pth'
missing/unexpected keys: [] []


In [8]:
face_encoder = FaceEncoder()
state_dict = torch.load(r"face2voice\checkpoints\face_encoder\facenet_checkpoint.pth")
face_encoder.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [9]:
face2voice = Face2VoiceModel(face_encoder=face_encoder, speaker_encoder=speaker_encoder)

In [10]:
face_transform = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [38]:
img = Image.open("resources\Della_Reese_5g5t3E-Xsi0_0000001.jpg").convert("RGB")
img = face_transform(img).unsqueeze(0)

In [50]:
tgt_emb = face2voice(img)

In [ ]:
spec = np.load("resources\A.J._Buckley_1zcIwhmdeo4_0000001.npz")
spec = torch.from_numpy(spec)

In [31]:
src_emb = speaker_encoder.encode_single(audio="resources\example_reference.mp3", input="audio", return_numpy=False)

OpenVoice version: v1
[(0.0, 58.8188125)]
after vad: dur = 58.81798185941043


c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\torch\functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\SpectralOps.cpp:880.)
  return _VF.stft(  # type: ignore[attr-defined]


In [52]:
tgt_emb = tgt_emb.detach().clone().requires_grad_(True).reshape(1, -1, 1)
src_emb = src_emb.detach().clone().requires_grad_(True).transpose(1, 2).squeeze(0).reshape(1, -1, 1)
print(tgt_emb.shape, src_emb.shape)

torch.Size([1, 256, 1]) torch.Size([1, 256, 1])


In [53]:
speaker_encoder.tone_color_converter.convert(audio_src_path="resources/example_reference.mp3", src_se=src_emb, tgt_se=tgt_emb, output_path="outputs/testwspec2.wav")

In [1]:
from face2voice.datasets.FaceVoiceDataset import FaceVoiceDataset
from torch.utils.data import DataLoader
from torchvision import transforms

test_dataset = FaceVoiceDataset(audio_base_path=r"C:\Users\user\foldered_extracted\wav", 
                                face_image_base_path=r'C:\Users\user\Desktop\tempo\results\voxceleb_processed\train\faces',
                                csv_path="resources/6_to_10_vox1_part_a.csv", split="train", 
                                transform_face=None)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Loaded train split:
  Total samples: 15284
  Unique speakers: 395
  Avg samples per speaker: 38.7


In [2]:
for batch in test_dataloader:
    print(batch)
    break

c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
c:\Users\user\Desktop\projects\face2voice\.venv\lib\site-packages\torch\functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\at

[Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\tempo\\results\\voxceleb_processed\\train\\faces\\Jonathan_Jackson.jpg'
[Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\tempo\\results\\voxceleb_processed\\train\\faces\\Melody_Thomas_Scott.jpg'
[Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\tempo\\results\\voxceleb_processed\\train\\faces\\S._Epatha_Merkerson.jpg'
[Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\tempo\\results\\voxceleb_processed\\train\\faces\\Samuel_L._Jackson.jpg'
[Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\tempo\\results\\voxceleb_processed\\train\\faces\\Sheryl_Lee_Ralph.jpg'


TypeError: 'NoneType' object is not subscriptable

In [3]:
print(sample["spectrogram"].shape, sample["face"].shape)
print(sample["speaker_name"])

torch.Size([1, 513, 599]) torch.Size([3, 112, 112])
Meat_Loaf


In [4]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

img = sample["face"].numpy().transpose(1, 2, 0) * 255

img = Image.fromarray(img.astype("uint8"))
img.show()

In [5]:
import os
fpath = r"C:\Users\user\Desktop\tempo\results\voxceleb_processed\train\faces"
for file in os.listdir(fpath):
    if ".jpg.jpg" in file:
        new_name = file.replace(".jpg.jpg", ".jpg")
        old = os.path.join(fpath, file)
        new = os.path.join(fpath, new_name)
        os.rename(old, new)